## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(r"C:\Users\celas\OneDrive\Desktop\Weather_Database\weatherpy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Norman Wells,CA,2022-08-11 23:34:53,65.2820,-126.8329,73.44,25,20,4.61,few clouds
1,1,Baruun-Urt,MN,2022-08-11 23:34:53,46.6806,113.2792,63.86,65,95,7.27,overcast clouds
2,2,Albany,US,2022-08-11 23:32:59,42.6001,-73.9662,80.51,54,60,1.01,broken clouds
3,3,Tuktoyaktuk,CA,2022-08-11 23:34:53,69.4541,-133.0374,53.60,82,75,11.50,broken clouds
4,4,Atar,MR,2022-08-11 23:34:54,20.5169,-13.0499,97.52,30,92,9.06,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your trip?'))
max_temp = float(input('What is the maximum temperature you would like for your trip?'))

What is the minimum temperature you would like for your trip?30
What is the maximum temperature you would like for your trip?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_preferred_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
cities_preferred_df.head()

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Norman Wells,CA,2022-08-11 23:34:53,65.2820,-126.8329,73.44,25,20,4.61,few clouds
1,1,Baruun-Urt,MN,2022-08-11 23:34:53,46.6806,113.2792,63.86,65,95,7.27,overcast clouds
2,2,Albany,US,2022-08-11 23:32:59,42.6001,-73.9662,80.51,54,60,1.01,broken clouds
3,3,Tuktoyaktuk,CA,2022-08-11 23:34:53,69.4541,-133.0374,53.60,82,75,11.50,broken clouds
5,5,Chiredzi,ZW,2022-08-11 23:34:54,-21.0500,31.6667,62.10,65,0,3.89,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
print(len(cities_preferred_df))
cities_preferred_df.count()

1292


Unnamed: 0             1292
City                   1292
Country                1278
Date                   1292
Lat                    1292
Lng                    1292
Max Temp               1292
Humidity               1292
Cloudiness             1292
Wind Speed             1292
Current Description    1292
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_preferred_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

1278


,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Norman Wells,CA,2022-08-11 23:34:53,65.2820,-126.8329,73.44,25,20,4.61,few clouds
1,1,Baruun-Urt,MN,2022-08-11 23:34:53,46.6806,113.2792,63.86,65,95,7.27,overcast clouds
2,2,Albany,US,2022-08-11 23:32:59,42.6001,-73.9662,80.51,54,60,1.01,broken clouds
3,3,Tuktoyaktuk,CA,2022-08-11 23:34:53,69.4541,-133.0374,53.60,82,75,11.50,broken clouds
5,5,Chiredzi,ZW,2022-08-11 23:34:54,-21.0500,31.6667,62.10,65,0,3.89,clear sky


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Norman Wells,CA,73.44,few clouds,65.2820,-126.8329,
1,Baruun-Urt,MN,63.86,overcast clouds,46.6806,113.2792,
2,Albany,US,80.51,broken clouds,42.6001,-73.9662,
3,Tuktoyaktuk,CA,53.60,broken clouds,69.4541,-133.0374,
5,Chiredzi,ZW,62.10,clear sky,-21.0500,31.6667,
6,Saldanha,ZA,55.45,overcast clouds,-33.0117,17.9442,
8,Mahebourg,MU,68.25,broken clouds,-20.4081,57.7000,
9,San Quintin,MX,75.94,overcast clouds,30.4833,-115.9500,
10,Ushuaia,AR,33.46,scattered clouds,-54.8000,-68.3000,
11,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")
        hotel_df.loc[index, "Hotel Name"] = np.nan
        

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
H

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Norman Wells,CA,73.44,few clouds,65.2820,-126.8329,Heritage Hotel
1,Baruun-Urt,MN,63.86,overcast clouds,46.6806,113.2792,Ахмадын амралт сувилал
3,Tuktoyaktuk,CA,53.60,broken clouds,69.4541,-133.0374,End of The Road Ltd.
5,Chiredzi,ZW,62.10,clear sky,-21.0500,31.6667,Clevers Hotel Chiredzi
6,Saldanha,ZA,55.45,overcast clouds,-33.0117,17.9442,Blue Bay Lodge
8,Mahebourg,MU,68.25,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
9,San Quintin,MX,75.94,overcast clouds,30.4833,-115.9500,Don Eddie's / Hotel - Restaurant
10,Ushuaia,AR,33.46,scattered clouds,-54.8000,-68.3000,Albatross Hotel
11,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,Keleti Beach Resort
12,Awjilah,LY,82.71,clear sky,29.1081,21.2869,المنزل


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
display(fig)

Figure(layout=FigureLayout(height='420px'))